In [33]:
import numpy as np
import random
import matplotlib.pyplot as plt
import tclab
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split 
import imageio

In [34]:
def generate_temperature_setpoints(initial_rest, initial_temp, min_interval, max_interval, min_temp, max_temp, count):
    start = 0
    end = initial_rest
    Tsp = np.zeros(count) * initial_temp
    while end <= count:
        start = end
        end += random.randint(min_interval, max_interval) 
        Tsp[start:end] = random.randint(min_temp, max_temp)
    return Tsp

In [35]:
def set_temperature_setpoints(x, y, count):
    Tsp = np.zeros(count)
    for i in range(len(x) - 1):
        Tsp[x[i]:x[i + 1]] = y[i]
    Tsp[x[len(x) - 1]:count] = y[len(y) - 1]
    return Tsp

In [36]:
def plot(data, x_label, y_label, label_size=14, ticks_size=12):
    plt.plot(data)
    plt.xlabel(x_label, size=label_size)
    plt.ylabel(y_label, size=label_size)
    plt.xticks(size=ticks_size)
    plt.yticks(size=ticks_size)

In [37]:
def run_tclab(run_time, Tsp, active_cont, passive_cont = None, Q2 = None, emulator=False, speedup=60):
    
    loops = int(run_time)
    
    T1 = np.zeros(loops)
    Q1 = np.zeros(loops)
    Qpass = np.zeros(loops)
    tm = np.zeros(loops)
    
    if emulator:
        TCLab = tclab.setup(connected=False, speedup=speedup)
    with TCLab() as lab:

        start_time = 0
        prev_time = -1

        i = 0
        for clock in tclab.clock(run_time):
            if i >= run_time:
                break
                
            t = clock
            dt = t - prev_time
            prev_time = t
            tm[i] = t - start_time

            T1[i] = lab.T1
            if passive_cont is not None:
                Qpass[i] = passive_cont.get_output(Tsp[i], T1[i], dt)
            Q1[i] = active_cont.get_output(Tsp[i], T1[i], dt)
            
            lab.Q1(Q1[i])
            if Q2 is not None:
                lab.Q2(Q2[i])
            
            print(f'{tm[i]:6.1f} {Tsp[i]:6.2f} {T1[i]:6.2f} {Q1[i]:6.2f} {Qpass[i]:6.2f}')
            
            i += 1
            
        return tm, T1, Q1, Qpass

In [38]:
def resume_run(lab, run_time, Tsp, active_cont, passive_cont = None, Q2 = None, emulator=False, speedup=60):
    
    loops = int(run_time)
    
    T1 = np.zeros(loops)
    Q1 = np.zeros(loops)
    Qpass = np.zeros(loops)
    tm = np.zeros(loops)
    
    start_time = 0
    prev_time = -1

    i = 0
    for clock in tclab.clock(run_time):
        if i >= run_time:
            break

        t = clock
        dt = t - prev_time
        prev_time = t
        tm[i] = t - start_time

        T1[i] = lab.T1
        if passive_cont is not None:
            Qpass[i] = passive_cont.get_output(Tsp[i], T1[i], dt)
        Q1[i] = active_cont.get_output(Tsp[i], T1[i], dt)

        lab.Q1(Q1[i])
        if Q2 is not None:
            lab.Q2(Q2[i])

        print(f'{tm[i]:6.1f} {T1[i]:6.2f} {Q1[i]:6.2f} {Qpass[i]:6.2f}')

        i += 1

    return tm, T1, Q1, Qpass

In [39]:
def select_best_features(X, y, threshold = 0.05):
    bestfeatures = SelectKBest(score_func=f_regression, k='all')
    fit = bestfeatures.fit(X,y)
    scores = fit.scores_
    total_score = sum(scores)
    
    features = list()
    
    for i in range(len(X.columns)):
        if scores[i] >= threshold * total_score:
            features.append(X.columns[i])
    
    plt.bar(x=X.columns,height=fit.scores_ / total_score);
    
    return X[features]

In [40]:
def create_train_and_test_data(X, y, s_x, s_y, window, test_size=0.2, shuffle=False):

        Xs = s_x.fit_transform(X)
        ys = s_y.fit_transform(y)

        X_lstm = []
        y_lstm = []
        for i in range(window, len(X)):
            X_lstm.append(Xs[i-window:i])
            y_lstm.append(ys[i])

        X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)
        return train_test_split(X_lstm, y_lstm, test_size=test_size, shuffle=shuffle)

In [41]:
def make_image_history(path, length, step, data):
    for i in range(0, length, step):
        colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
        plt.figure(figsize=(10,4))
        for label, item in data.items():
            plt.plot(item[:i], f'-{colors.pop(0)}', label=label)
        plt.legend(loc='upper right',fontsize=14)
        plt.ylim((0,100))
        plt.xlabel('Time (s)',size=14)
        plt.xticks(size=12)
        plt.yticks(size=12)

        num = str(i).zfill(len(str(length)))
        filename=path.format(num)
        plt.savefig(filename)
        plt.close()

In [42]:
def read_image_history(path, length, step):
    images = []
    for i in range(0,length,step):
        num = str(i).zfill(len(str(length)))
        filename=path.format(num)
        images.append(imageio.imread(filename))
    return images

In [43]:
def multiplot(data, xlabel='x', ylabel='y', label_size=14, ticks_size=12):
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
    for label, item in data.items():
        plt.plot(item, f'-{colors.pop(0)}', label=label)
    plt.xlabel(xlabel, size=label_size)
    plt.ylabel(ylabel, size=label_size)
    plt.xticks(size=12)
    plt.yticks(size=12)
    plt.legend()